# Angelic Search 

Search using angelic semantics (is a hierarchical search), where the agent chooses the implementation of the HLA's. <br>
The algorithms input is: problem, hierarchy and initialPlan
-  problem is of type Problem 
-  hierarchy is a dictionary consisting of all the actions. 
-  initialPlan is an approximate description(optimistic and pessimistic) of the agents choices for the implementation. <br>
   It is a nested list, containing sequence a of actions with their optimistic and pessimistic
   description 

In [65]:
from planning import * 

The Angelic search algorithm consists of three parts. 
-  Search using angelic semantics
-  Decompose
-  a search in the space of refinements, in a similar way with hierarchical search

### Searching using angelic semantics
-  Find the reachable set (optimistic and pessimistic) of the sequence of angelic HLA in initialPlan
  -    If the optimistic reachable set doesn't intersect the goal, then there is no solution
  -    If the pessimistic reachable set intersects the goal, then we call decompose, in order to find the sequence of actions that lead us to the goal. 
  -    If the optimistic reachable set intersects the goal, but the pessimistic doesn't we do some further refinements, in order to see if there is a sequence of actions that achieves the goal. 
  
### Search in space of refinements
-  We create a search tree, that has root the action and children it's refinements
-  We extend frontier by adding each refinement, so that we keep looping till we find all primitive actions
-  If we achieve that we return the path of the solution (search tree), else there is no solution and we return None.

  



### Decompose 
-  Finds recursively the sequence of states and actions that lead us from initial state to goal.
-  For each of the above actions we find their refinements,if they are not primitive, by calling the angelic_search function. 
   If there are not refinements return None
   


## Example

Suppose that somebody wants to get to the airport. 
The possible ways to do so is either get a taxi, or drive to the airport. <br>
Those two actions have some preconditions and some effects. 
If you get the taxi, you need to have cash, whereas if you drive you need to have a car. <br>
Thus we define the following hierarchy of possible actions.

##### hierarchy

In [66]:
library = {
        'HLA': ['Go(Home,SFO)', 'Go(Home,SFO)', 'Drive(Home, SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)', 'Taxi(Home, SFO)'],
        'steps': [['Drive(Home, SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)'], ['Taxi(Home, SFO)'], [], [], []],
        'precond': [['At(Home)', 'Have(Car)'], ['At(Home)'], ['At(Home)', 'Have(Car)'], ['At(SFOLongTermParking)'], ['At(Home)']],
        'effect': [['At(SFO)'], ['At(SFO)'], ['At(SFOLongTermParking)'], ['At(SFO)'], ['At(SFO)'], ['~At(Home)'], ['~At(Home)'], ['~At(Home)'], ['~At(SFOLongTermParking)'], ['~At(Home)']]
        }


the possible actions are the following:

In [67]:

go_SFO = HLA('Go(Home,SFO)', precond='At(Home)', effect='At(SFO) & ~At(Home)')
taxi_SFO = HLA('Taxi(Home,SFO)', precond='At(Home) & Have(Cash)', effect='At(SFO) & ~At(Home)')
drive_SFOLongTermParking = HLA('Drive(Home,SFOLongTermParking)', precond='At(Home) & Have(Car)', effect='At(SFOLongTermParking) & ~At(Home)')
shuttle_SFO = HLA('Shuttle(SFOLongTermParking,SFO)', precond = 'At(SFOLongTermParking)', effect = 'At(SFO) & ~At(SFOLongTermParking)')

Suppose that (our preconditionds are that) we are Home and we have cash and car and  our goal is to get to SFO and maintain our cash, and our possible actions are the above. <br>
##### Then our problem is: 

In [68]:
prob = Problem('At(Home) & Have(Cash) & Have(Car)', 'At(SFO) & Have(Cash)', [go_SFO, taxi_SFO, drive_SFOLongTermParking,shuttle_SFO])

An agent gives us some approximate information about the plan we will follow: <br>
(initialPlan is a list with two nested lists, containing the optimistic and pessimistic descriptions respectively, of the sequence of angelic HLA's)
##### InitialPlan

In [69]:
angelic_opt_description = Angelic_HLA('Go(Home, SFO)', precond = 'At(Home)', effect ='$+At(SFO) & $-At(Home)' ) 
angelic_pes_description = Angelic_HLA('Go(Home, SFO)', precond = 'At(Home)', effect ='$+At(SFO) & ~At(Home)' ) 

initialPlan = [Angelic_Node(prob.init, None, [angelic_opt_description], [angelic_pes_description])] 


We want to find the optimistic and pessimistic reachable set of initialPlan_1 when applied to the problem:
##### Optimistic/Pessimistic reachable set

In [70]:
opt_reachable_set = Problem.reach_opt(prob.init, initialPlan[0])
pes_reachable_set = Problem.reach_pes(prob.init, initialPlan[0])
print([x for y in opt_reachable_set.keys() for x in opt_reachable_set[y]], '\n')
print([x for y in pes_reachable_set.keys() for x in pes_reachable_set[y]])


[[At(Home), Have(Cash), Have(Car)], [Have(Cash), Have(Car), At(SFO), NotAt(Home)], [Have(Cash), Have(Car), NotAt(Home)], [At(Home), Have(Cash), Have(Car), At(SFO)], [At(Home), Have(Cash), Have(Car)]] 

[[At(Home), Have(Cash), Have(Car)], [Have(Cash), Have(Car), At(SFO), NotAt(Home)], [Have(Cash), Have(Car), NotAt(Home)]]


##### Refinements

In [71]:
for sequence in Problem.refinements(go_SFO, prob, library):
    print([x.name for x in sequence])
    print([x.__dict__ for x in sequence ], '\n')

['Drive', 'Shuttle']
[{'precond': [At(Home), Have(Car)], 'name': 'Drive', 'completed': False, 'args': (Home, SFOLongTermParking), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}, {'precond': [At(Home), Have(Car)], 'name': 'Shuttle', 'completed': False, 'args': (SFOLongTermParking, SFO), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}] 

['Taxi']
[{'precond': [At(Home)], 'name': 'Taxi', 'completed': False, 'args': (Home, SFO), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}] 



Run the angelic search
##### Top level call

In [72]:
plan= Problem.angelic_search(prob, library, initialPlan)
print([(x.name, x.args) for x in plan], '\n')
print([x.__dict__ for x in plan])

[('Drive', (Home, SFOLongTermParking)), ('Shuttle', (SFOLongTermParking, SFO))] 

[{'precond': [At(Home), Have(Car)], 'name': 'Drive', 'completed': False, 'args': (Home, SFOLongTermParking), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}, {'precond': [At(Home), Have(Car)], 'name': 'Shuttle', 'completed': False, 'args': (SFOLongTermParking, SFO), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}]


## Example 2
(using decompose)

In [94]:
library = {
        'HLA': ['Go(Home,SFO)', 'Go(Home,SFO)', 'Drive(Home, SFOLongTermParking)', 'Get(Ticket)' , 'Shuttle(SFOLongTermParking, SFO)', 'Taxi(Home, SFO)'],
        'steps': [['Drive(Home, SFOLongTermParking)', 'Shuttle(SFOLongTermParking, SFO)'], ['Taxi(Home, SFO)'], ['Get(Ticket)'],[], [], []],
        'precond': [['At(Home)', 'Have(Car)'], ['At(Home)'], ['At(Home)', 'Have(Car)'],['At(SFOLongTermPark)',' Have(Cash)'], ['At(SFOLongTermParking) & Have(Ticket)'], ['At(Home)']],
        'effect': [['At(SFO)'], ['At(SFO)'], ['At(SFOLongTermParking)'], ['Have(Ticket)'] ,['At(SFO)'], ['At(SFO)'], ['~At(Home)'], ['~At(Home)'], ['~At(Home)'], ['~Have(Cash)'] ,['~At(SFOLongTermParking)'], ['~At(Home)','~Have(Cash)']]
        }

possible actions

In [95]:
go_SFO = HLA('Go(Home,SFO)', precond='At(Home)', effect='At(SFO) & ~At(Home)')
taxi_SFO = HLA('Taxi(Home,SFO)', precond='At(Home) & Have(Cash)', effect='At(SFO) & ~At(Home)')
drive_SFOLongTermParking = HLA('Drive(Home,SFOLongTermParking)', precond='At(Home) & Have(Car)', effect='At(SFOLongTermParking) & ~At(Home)')
get_ticket = HLA('Get(Ticket)', precond = 'At(SFOLongTermPark) & Have(Cash)', effect = 'Have(Ticket) & ~Have(Cash)')
shuttle_SFO = HLA('Shuttle(SFOLongTermParking,SFO)', precond = 'At(SFOLongTermParking)', effect = 'At(SFO) & ~At(SFOLongTermParking)')

In [93]:
prob = Problem('At(Home) & Have(Cash) & Have(Car)', 'At(SFO)', [go_SFO, taxi_SFO, drive_SFOLongTermParking,get_ticket,shuttle_SFO])
initialPlan = [Angelic_Node(prob.init, None, [angelic_opt_description], [angelic_pes_description])]
plan = Problem.angelic_search(prob, library, initialPlan)
print([(x.name, x.args) for x in plan], '\n')
print([x.__dict__ for x in plan])

[('Shuttle', (SFOLongTermParking, SFO)), ('Get', (Ticket,))] 

[{'precond': [At(Home), Have(Car)], 'name': 'Shuttle', 'completed': False, 'args': (SFOLongTermParking, SFO), 'duration': 0, 'uses': {}, 'effect': [At(SFO)], 'consumes': {}}, {'precond': [At(Home), Have(Car)], 'name': 'Get', 'completed': False, 'args': (Ticket,), 'duration': 0, 'uses': {}, 'effect': [At(SFOLongTermParking)], 'consumes': {}}]
